# Dataset Pre-Processing

In this notebook, we present a series of steps to illustrate the process of cleaning out a dataset before using it for a learnng task related to patterns-recognition.

For this purpose, we are going to use the **1.6 million UK traffic accidents** dataset that can be downloaded from the next link: https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales

From its documentation, this dataset contains the most relevant information for police reported accidents in the UK between 2005 and 2014, although 2008 is missing. Each of the 1.6 million accidents reported in this dataset is described with 33 features (33 columns for each instance).

We will attempt to visualize and understand the most relevant contents of the dataset while implementing improvements that can make it more suitable for a learning task. It is worth mentioning that the specific type of pre-processing to apply depends on the application and type of model we desire to implement. We will comment on certain actions that could be taken but that are not necessarily applied to keep the data somehow generic.


## Loading the Dataset

In [ ]:
# Start by importing relevant python modules
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Load data from the 3 datasets and merge into a single data-frame
# Note: Specifying type for one of the columns since it seems to have mixed datatypes
df_05_07 = pd.read_csv('dataset/raw/accidents_2005_to_2007.csv', dtype={'LSOA_of_Accident_Location': 'string'})
df_09_11 = pd.read_csv('dataset/raw/accidents_2009_to_2011.csv', dtype={'LSOA_of_Accident_Location': 'string'})
df_12_14 = pd.read_csv('dataset/raw/accidents_2012_to_2014.csv', dtype={'LSOA_of_Accident_Location': 'string'})
df_accidents_05_14 = pd.concat([df_05_07, df_09_11, df_12_14], ignore_index=True)

In [ ]:
# Confirm values have been concatenated vertically by printing the DF size
print("df_accidents_05_14 shape is: {}".format(df_accidents_05_14.shape))

From the printed text above, We can observe how the whole raw dataset has a little more than 1.5 million instances and not quite 1.6 million.

In [ ]:
# Print head to visualize initial column values
df_accidents_05_14.head()

## Removing Replicated Instances
Even though the instances are supposed to be unique, some of them were actually replicated from file to file.
See this: https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales/discussion/51835

We can confirm this by printing the count for the most repeated values in the *'Accident_Index'* column. As it can be seen below, there is a chance that the same accidents are being replicated multiple times.

In [ ]:
# Print 10 most frequent IDs and their count
print(df_accidents_05_14['Accident_Index'].value_counts()[:10])

As recommended in the discussion linked above, we will remove the replicated items when they have the same value for multiple properties. This because there is a disclaimer on the page of the dataset indicating that the *'Accident_Index'* property may not be formatted correctly in all the cases and we should not trust it 100% as unique.

In [ ]:
relevant_vars=['Accident_Index','Date','LSOA_of_Accident_Location','Time','Longitude','Latitude']
df_accidents_05_14 = df_accidents_05_14.drop_duplicates(subset=relevant_vars, keep='first')

In [ ]:
# Print the new shape
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))

### Removing *'Accident_Index'* Column
As it was discussed above, we cannot trust this column to be formatted correctly and be unique for each of the reported accidents. Besides, since this is an identifier, it would not offer any valuable insight for a training algorithm. We will go ahead and remove it then.

In [ ]:
# Drop 'Accident_Index' column
df_accidents_05_14 = df_accidents_05_14.drop(columns=['Accident_Index'])
# Print the new shape and head
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))
df_accidents_05_14.head()

## Reviewing Columns Data
We will now review the data provided for each of the columns and identify improvements before using them on a training model. We will attempt to visualize the data, identify its usefulness, detect missing elements, identify outliers, group or join elements, transform, normalize, etc.


### Geo-Localization Columns
The first 4 columns contain localization information for the accident. The first 2 columns (*'Location_Easting_OSGR'* and *'Location_Northing_OSGR'*) can define a unique location by themselves and are specific for the UK. Considering this, we can identify the *'Longitude'* and *'Latitude'* columns as redundant and we can apply a dimmensionality reduction by removing them.

In [ ]:
# Drop 'Longitude', 'Latitude' columns
df_accidents_05_14 = df_accidents_05_14.drop(columns=['Longitude', 'Latitude'])
# Print the new shape and head
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))
df_accidents_05_14.head()

As it can be seen below, the two location columns do have missing values:

In [ ]:
df_accidents_05_14[df_accidents_05_14['Location_Easting_OSGR'].isna() | df_accidents_05_14['Location_Northing_OSGR'].isna()]

Since the values represent physical locations, it may not make sense to fill these values with other and we opt for removing these 101 instances.

In [ ]:
df_accidents_05_14.dropna(subset=['Location_Easting_OSGR', 'Location_Northing_OSGR'], inplace=True)
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))

Now let's visualize in a scatter the location columns:

In [ ]:
df_accidents_05_14.plot.scatter(x='Location_Easting_OSGR', y='Location_Northing_OSGR')

At this point and depending on the application or desired model, we may filter the locations to only include certain regions or areas. However in this case we are going to keep them all. 
One final step we will make with this location data is reducing their range. These location values are high numbers, while the rest of the columns are not, and also to avoid potential overflow in calculations when implementing a model. For this purpose we will apply a scale of 1e-6.

In [ ]:
location_scale = 1e-6
df_accidents_05_14['Location_Easting_OSGR'] = df_accidents_05_14['Location_Easting_OSGR'] * location_scale
df_accidents_05_14['Location_Northing_OSGR'] = df_accidents_05_14['Location_Northing_OSGR'] * location_scale
# Visualize updated location data
df_accidents_05_14.plot.scatter(x='Location_Easting_OSGR', y='Location_Northing_OSGR')

### *'Police_Force'* Column
This column indicates an identifier for the police force responsible of reporting the accident. As we can see below this column does not have missing items.

In [ ]:
df_accidents_05_14[df_accidents_05_14['Police_Force'].isna()]

From the data visualization below, we can see the data is in an acceptable range and there is not any visible outlier. We will keep this data as it is.

In [ ]:
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['Police_Force'])
plt.xlabel("index")
plt.ylabel("Police_Force")

### *'Accident_Severity'* Column
This column indicates the severity of the accident in 3 possible values: 

1- High Severity.

2- Mid Severity.

3- Low Severity.

As we can see below this column does not have missing items.

In [ ]:
df_accidents_05_14[df_accidents_05_14['Accident_Severity'].isna()]

Since the severity assignments do not correspond with the numerical value, we choose to change the values to follow the next more logical representation:
1- Low Severity.

2- Mid Severity.

3- High Severity.

In [ ]:
# Temporarily save 3 as 0
df_accidents_05_14.loc[df_accidents_05_14['Accident_Severity'] == 3, 'Accident_Severity'] = 0
# Save 1 as 3
df_accidents_05_14.loc[df_accidents_05_14['Accident_Severity'] == 1, 'Accident_Severity'] = 3
# Save 0 as 1
df_accidents_05_14.loc[df_accidents_05_14['Accident_Severity'] == 0, 'Accident_Severity'] = 1

As we can observe on the histogram below, most of the instances represent low-severity accidents. Depending on the application, we may desire to balance these classes distribution by removing some of the instances of low-severity.

In [ ]:
df_accidents_05_14.hist(column='Accident_Severity', bins=3)

### *'Number_of_Vehicles'* Column
This column indicates the number of vehicles involved in the accident. As we can see below this column does not have missing items.

In [ ]:
df_accidents_05_14[df_accidents_05_14['Number_of_Vehicles'].isna()]

From the data visualization below, we can identify there are a few outliers on the data with not enough instances to be representative for the model training. 

In [ ]:
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['Number_of_Vehicles'])
plt.xlabel("index")
plt.ylabel("Number_of_Vehicles")

Considering the above, we will remove the outlisers that are not within the +-3 standard deviations from mean. With this approach only the values of 1 to 4 are maintained for this column and the number of instances is reduced to 1461460.

In [ ]:
# Keep only the ones that are within +3 to -3 standard deviations (rounded)
nv_column = df_accidents_05_14['Number_of_Vehicles']
nv_mean = nv_column.mean()
nv_std_dev = nv_column.std()
print("Number_of_Vehicles mean = {}".format(nv_mean))
print("Number_of_Vehicles std deviation = {}".format(nv_std_dev))
deviations = np.abs(nv_column - nv_mean)
df_accidents_05_14 = df_accidents_05_14[nv_column <= np.round(nv_mean + 3 * nv_std_dev)]

# Plot Number_of_Vehicles again
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['Number_of_Vehicles'])
plt.xlabel("index")
plt.ylabel("Number_of_Vehicles Updated")
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))

### *'Number_of_Casualties'* Column
This column indicates the number of casualities as a consequence of the accident. As we can see below this column does not have missing items.

In [ ]:
df_accidents_05_14[df_accidents_05_14['Number_of_Casualties'].isna()]

From the data visualization below and as in the previous case, we can identify there are a few outliers on the data.

In [ ]:
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['Number_of_Casualties'])
plt.xlabel("index")
plt.ylabel("Number_of_Casualties")

Considering the above, we will remove the outlisers that are not within the +-3 standard deviations from mean. With this approach only the values of 1 to 4 are maintained for this column and the number of instances is reduced to 1447801.

In [ ]:
# Keep only the ones that are within +3 to -3 standard deviations (rounded)
nc_column = df_accidents_05_14['Number_of_Casualties']
nc_mean = nc_column.mean()
nc_std_dev = nc_column.std()
print("Number_of_Casualties mean = {}".format(nc_mean))
print("Number_of_Casualties std deviation = {}".format(nc_std_dev))
deviations = np.abs(nc_column - nc_mean)
df_accidents_05_14 = df_accidents_05_14[nc_column <= np.round(nc_mean + 3 * nc_std_dev)]

# Plot Number_of_Casualties again
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['Number_of_Casualties'])
plt.xlabel("index")
plt.ylabel("Number_of_Casualties Updated")
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))

### *'Date'*, *'Day_of_Week'* and *'Time'* Columns
These columns locate in time the presence of the accident. Start by determining if they have missing values:


In [ ]:
print("Missing values in Date: {}".format(df_accidents_05_14[df_accidents_05_14['Date'].isna()].shape[0]))
print("Missing values in Day_of_Week: {}".format(df_accidents_05_14[df_accidents_05_14['Day_of_Week'].isna()].shape[0]))
print("Missing values in Time: {}".format(df_accidents_05_14[df_accidents_05_14['Time'].isna()].shape[0]))

In [ ]:
# Drop rows with missing values for Time
df_accidents_05_14.dropna(subset=['Time'], inplace=True)
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))

Since the *'Date'* as it is will not represent anything for a mathematical model, we will break it into 3 separate columns: *'Month_Day'*, *'Month'* and *'Year'*.

In [ ]:
# Start by converting Date column to datetime type
df_accidents_05_14['Date'] = pd.to_datetime(df_accidents_05_14['Date'], format='%d/%m/%Y')

# Create new separate columns
df_accidents_05_14['Month_Day'] = df_accidents_05_14['Date'].dt.day
df_accidents_05_14['Month'] = df_accidents_05_14['Date'].dt.month
df_accidents_05_14['Year'] = df_accidents_05_14['Date'].dt.year

# Drop 'Date' column
df_accidents_05_14 = df_accidents_05_14.drop(columns=['Date'])
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))


Since the *'Time'* as it is will not represent anything for a mathematical model, we will break it into 2 separate columns: *'Hour'* and *'Minute'*.

In [ ]:
# Start by converting Time column to datetime type
df_accidents_05_14['Time'] = pd.to_datetime(df_accidents_05_14['Time'], format='%H:%M')

# Create new separate columns
df_accidents_05_14['Hour'] = df_accidents_05_14['Time'].dt.hour
df_accidents_05_14['Minute'] = df_accidents_05_14['Time'].dt.minute

# Drop 'Time' column
df_accidents_05_14 = df_accidents_05_14.drop(columns=['Time'])
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))

In [ ]:
df_accidents_05_14

### 'Local_Authority_(District)' Column
This column represents the identifier for the district authority. As we can see below this column does not have missing items.

In [ ]:
df_accidents_05_14[df_accidents_05_14['Local_Authority_(District)'].isna()]

From the plot below, we do not see any clear outlier. We will just rename the column to avoid the parenthesis in the name.

Since these are categories, one option here may have been converting to separate columns using one-hot encodings, however in this case we are dealing with almost 1000 types which makes it inconvenient.

In [ ]:
df_accidents_05_14 = df_accidents_05_14.rename(columns={'Local_Authority_(District)': 'Local_Authority_District'})

plt.scatter(df_accidents_05_14.index, df_accidents_05_14['Local_Authority_District'])
plt.xlabel("index")
plt.ylabel("Local_Authority_District")

### 'Local_Authority_(Highway)' Column
This column represents the identifier for the highway authority. As we can see below this column does not have missing items. Also rename the column to avoid the parenthesis in the name.

In [ ]:
df_accidents_05_14 = df_accidents_05_14.rename(columns={'Local_Authority_(Highway)': 'Local_Authority_Highway'})
df_accidents_05_14[df_accidents_05_14['Local_Authority_Highway'].isna()]

Since this column also represents categories, we will convert it into this type for simpler manipulation.The same as before, one-hot encodings is not as practical since we are dealing with more than 200 cateogires.

In [ ]:
df_accidents_05_14['Local_Authority_Highway'] = df_accidents_05_14['Local_Authority_Highway'].astype('category')
# If we desired to have the codes as numbers
# df_accidents_05_14['Local_Authority_Highway'] = df_accidents_05_14['Local_Authority_Highway'].cat.codes

In [ ]:
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['Local_Authority_Highway'].cat.codes)
plt.xlabel("index")
plt.ylabel("Local_Authority_Highway (code)")

### *'1st_Road_Class'* and *'2nd_Road_Class'* Columns
From the dataset documentation, it is not very clear on what these columns represent. As it can be seen below, they does not have any missing values and contain 6 potential values. It may be wroth applying one-hot encodings to these columns after knowing what each of the categories represent.

In [ ]:
print("Missing values in 1st_Road_Class: {}".format(df_accidents_05_14[df_accidents_05_14['1st_Road_Class'].isna()].shape[0]))
print("Missing values in 2nd_Road_Class: {}".format(df_accidents_05_14[df_accidents_05_14['2nd_Road_Class'].isna()].shape[0]))

In [ ]:
df_accidents_05_14.hist(column='1st_Road_Class', bins=6)
df_accidents_05_14.hist(column='2nd_Road_Class', bins=6)

### *'1st_Road_Number'* and *'2nd_Road_Number'* Columns
First and second number for the road where the accident happened. There are not missing items and we will keep them as they are.

In [ ]:
print("Missing values in 1st_Road_Number: {}".format(df_accidents_05_14[df_accidents_05_14['1st_Road_Number'].isna()].shape[0]))
print("Missing values in 2nd_Road_Number: {}".format(df_accidents_05_14[df_accidents_05_14['2nd_Road_Number'].isna()].shape[0]))

In [ ]:
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['1st_Road_Number'])
plt.xlabel("index")
plt.ylabel("1st_Road_Number")

In [ ]:
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['2nd_Road_Number'])
plt.xlabel("index")
plt.ylabel("2nd_Road_Number")

### *'Road_Type'* Column
This column indicates the type of road where the accident was presented. As it can be seen below, we have no missing values:

In [ ]:
df_accidents_05_14[df_accidents_05_14['Road_Type'].isna()]

From the plot below, we can observe that there are five type of roads defined and an Unknown category

In [ ]:
# Plot by index
plt.scatter(df_accidents_05_14.index, df_accidents_05_14['Road_Type'])
plt.xlabel("index")
plt.ylabel("Road_Type")

# Count by each type
df_accidents_05_14['Road_Type'].value_counts()

This is a good oportunity to apply one-hot encodings over this column. We can only keep the the rows with the values with more instances and delete the rows with the *'Unknown'* column.

In [ ]:
# Get new columns from one-hot encodings and drop Unknown
one_hot = pd.get_dummies(df_accidents_05_14['Road_Type'])
one_hot.drop(columns=['Unknown'], inplace=True)

# Rename Columns of one-hot
one_hot.rename(columns={'Dual carriageway': 'Road_Dual_Carriageway',
                       'One way street': 'Road_One_Way_Street',
                       'Roundabout': 'Road_Roundabout',
                       'Single carriageway': 'Road_Single_Carriageway',
                       'Slip road': 'Road_Slip'}, 
               inplace=True)

# Add one-hot to original df and drop Road_Type column
df_accidents_05_14 = df_accidents_05_14.join(one_hot)
df_accidents_05_14.drop(columns=['Road_Type'], inplace=True)


In [ ]:
df_accidents_05_14

### *'Speed_limit'* Column
Speed limit for the road where the accident happened. There are not missing items and we will keep the data as it is.

In [ ]:
df_accidents_05_14[df_accidents_05_14['1st_Road_Number'].isna()]

In [ ]:
df_accidents_05_14.hist(column='Speed_limit', bins=10)

### *'Junction_Detail'* and *'Junction_Control'* Columns
As it can be seen below, there is a very high number of instances which do not have a value for these columns. Considering this, we will go ahead and get them completely deleted.

In [ ]:
print("Missing values in Junction_Detail: {}".format(df_accidents_05_14[df_accidents_05_14['Junction_Detail'].isna()].shape[0]))
print("Missing values in Junction_Control: {}".format(df_accidents_05_14[df_accidents_05_14['Junction_Control'].isna()].shape[0]))

In [ ]:
df_accidents_05_14.drop(columns=['Junction_Detail', 'Junction_Control'], inplace=True)
print("df_accidents_05_14 new shape is: {}".format(df_accidents_05_14.shape))

### *'Pedestrian_Crossing-Human_Control'* Column

This column contains a string indicating if there is human control for pedestrial crossing close to the site of the accident. Below can see that there are a few missing items that we proceed to remove.

In [ ]:
df_accidents_05_14 = df_accidents_05_14.rename(columns={'Pedestrian_Crossing-Human_Control': 'Pedestrian_Crossing_Human_Control'})
print("Missing values in Pedestrian_Crossing_Human_Control: {}".format(df_accidents_05_14[df_accidents_05_14['Pedestrian_Crossing_Human_Control'].isna()].shape[0]))
df_accidents_05_14.dropna(subset=['Pedestrian_Crossing_Human_Control'], inplace=True)

Below we identify the type of classes for this column. As it can be seen below there are only 3 type of classes, which means we can apply one-hot encodings for these.

In [ ]:
# Count by each type
df_accidents_05_14['Pedestrian_Crossing_Human_Control'].value_counts()

In [ ]:
# Get new columns from one-hot encodings
one_hot = pd.get_dummies(df_accidents_05_14['Pedestrian_Crossing_Human_Control'])

# Rename Columns of one-hot
one_hot.rename(columns={'None within 50 metres': 'Pedestrian_Crossing_Control_None',
                       'Control by other authorised person': 'Pedestrian_Crossing_Control_Authorized_Person',
                       'Control by school crossing patrol': 'Pedestrian_Crossing_Control_School_Patrol'}, 
               inplace=True)

# Add one-hot to original df and drop Pedestrian_Crossing_Human_Control column
df_accidents_05_14 = df_accidents_05_14.join(one_hot)
df_accidents_05_14.drop(columns=['Pedestrian_Crossing_Human_Control'], inplace=True)

In [ ]:
df_accidents_05_14

### *'Pedestrian_Crossing-Physical_Facilities'* Column

This column contains a string indicating if there is a facility for pedestrial crossing close to the site of the accident. Below can see that there are a few missing items that we proceed to remove.

In [ ]:
df_accidents_05_14 = df_accidents_05_14.rename(columns={'Pedestrian_Crossing-Physical_Facilities': 'Pedestrian_Crossing_Physical_Facilities'})
print("Missing values in Pedestrian_Crossing_Physical_Facilities: {}".format(df_accidents_05_14[df_accidents_05_14['Pedestrian_Crossing_Physical_Facilities'].isna()].shape[0]))
df_accidents_05_14.dropna(subset=['Pedestrian_Crossing_Physical_Facilities'], inplace=True)

Below we identify the type of classes for this column. As it can be seen below there are only 6 types of classes, which means we can apply one-hot encodings for these.

In [ ]:
# Count by each type
df_accidents_05_14['Pedestrian_Crossing_Physical_Facilities'].value_counts()

In [ ]:
# Get new columns from one-hot encodings
one_hot = pd.get_dummies(df_accidents_05_14['Pedestrian_Crossing_Physical_Facilities'])

# Rename Columns of one-hot
one_hot.rename(columns={'No physical crossing within 50 meters': 'Pedestrian_Crossing_Fac_None',
                       'Pedestrian phase at traffic signal junction': 'Pedestrian_Crossing_Fac_Ped_Phase_at_Traffic',
                       'non-junction pedestrian crossing': 'Pedestrian_Crossing_Fac_NonJun_Crossing',
                       'Zebra crossing': 'Pedestrian_Crossing_Fac_Zebra',
                       'Central refuge': 'Pedestrian_Crossing_Fac_Central_Refuge',
                       'Footbridge or subway': 'Pedestrian_Crossing_Fac_Footbridge'}, 
               inplace=True)

# Add one-hot to original df and drop Pedestrian_Crossing_Physical_Facilities column
df_accidents_05_14 = df_accidents_05_14.join(one_hot)
df_accidents_05_14.drop(columns=['Pedestrian_Crossing_Physical_Facilities'], inplace=True)

In [ ]:
df_accidents_05_14.dtypes